In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [14]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
import nltk
from nltk import *
from nltk.corpus import wordnet
#nltk.download('wordnet')

In [5]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [8]:
print('train shape:',train.shape)
print("test shape:",test.shape)

train shape: (7613, 5)
test shape: (3263, 4)


In [9]:
def replace(txt):
    return re.sub(r'@\w+','',txt)

def hashtags(txt):
    return " ".join(re.findall(r'#(\w+)',txt))

#https://stackoverflow.com/questions/39634222/is-there-a-way-to-remove-proper-nouns-from-a-sentence-using-python
#def removeNouns(txt):
#    tagged_sentence = nltk.tag.pos_tag(txt.split())
#    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
#   return ' '.join(edited_sentence)
        

#https://stackoverflow.com/questions/54396405/how-can-i-preprocess-nlp-text-lowercase-remove-special-characters-remove-numb
def nltkpreprocess(txt):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(txt)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

def preprocess(df):
    df.text=df.text.str.lower() 
    df.text=df.text.apply(replace)
    df['hashtags']=df.text.apply(hashtags)
    df.text=df.text.replace(r'[^a-z ]+', '', regex=True)
    df.text=df.text.replace(r'http\S+', '', regex=True)
#    df.text=df.text.apply(removeNouns)
    df.text=df.text.apply(nltkpreprocess)
    return df
    

In [28]:
lemmatizer.lemmatize('bats')

'bat'

In [10]:
train=preprocess(train)
test=preprocess(test)

In [11]:
train.head()

,id,keyword,location,text,target,hashtags
0,1,NaN,NaN,deeds reason earthquake may allah forgive,1,earthquake
1,4,NaN,NaN,forest fire near ronge sask canada,1,
2,5,NaN,NaN,residents asked shelter place notified officer...,1,
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1,wildfires
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1,alaska wildfires


In [12]:
test.head()

,id,keyword,location,text,hashtags
0,0,NaN,NaN,happened terrible car crash,
1,2,NaN,NaN,heard earthquake different cities stay safe ev...,earthquake
2,3,NaN,NaN,forest fire spot pond geese fleeing across str...,
3,9,NaN,NaN,apocalypse lighting spokane wildfires,spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills china taiwan,


In [2]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [3]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True);

In [13]:
train.head()

,id,keyword,location,text,target,hashtags
0,1,NaN,NaN,deeds reason earthquake may allah forgive,1,earthquake
1,4,NaN,NaN,forest fire near ronge sask canada,1,
2,5,NaN,NaN,residents asked shelter place notified officer...,1,
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1,wildfires
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1,alaska wildfires


In [14]:
len(model.word_vec('Hi'))

300

In [15]:
def w2v_features(df,col):
    vecs=[]
    for text in list(df['text']):
        words = text.split() 
        mean_vec = np.zeros(300)
        for word in words:
            try:
                vec = model.word_vec(word)
            except:
                vec = np.zeros(300)
            mean_vec=mean_vec+(vec)
        mean_vec = mean_vec/(len(words)+1)
        vecs.append(mean_vec)
    return pd.DataFrame(vecs)
    

In [16]:
def new_features(df):
    features=w2v_features(df,'text')
    hashfeat=w2v_features(df,'hashtags')
    newdf=pd.concat([df,features],axis=1)
    newdf=pd.concat([newdf,hashfeat],axis=1)
    newdf.drop(columns=['location','text','keyword','hashtags'],inplace=True)
    return newdf 

In [17]:
newtrain=new_features(train)
newtest=new_features(test)

In [18]:
newtrain.head()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,1,1,0.135219,0.064479,0.160296,0.108747,-0.034249,0.061297,0.157715,-0.025792,...,-0.070312,0.077009,-0.091518,-0.051426,0.013463,0.026117,-0.039612,-0.081839,-0.040030,0.126675
1,4,1,0.047224,-0.008510,-0.016868,0.059884,0.113421,-0.031564,-0.000872,-0.084630,...,-0.020578,-0.011143,-0.026367,0.083801,0.039760,0.015172,-0.119001,-0.004708,0.070382,0.003789
2,5,1,0.004415,0.020355,0.080750,-0.049952,-0.053772,-0.091888,0.021645,-0.014414,...,-0.118062,-0.044556,-0.148926,0.010712,0.107188,0.043055,-0.078309,0.032735,0.011322,-0.030111
3,6,1,0.147705,-0.081909,0.012434,0.182460,0.022095,-0.066866,-0.051200,-0.024693,...,0.147025,-0.043806,-0.019053,0.020508,0.226772,0.053855,-0.189000,0.028041,0.008161,-0.039830
4,7,1,0.060986,0.026184,-0.059290,0.069708,-0.013550,0.007294,0.020361,-0.146606,...,0.058154,-0.147144,-0.058820,0.061597,0.038232,-0.025232,0.017001,-0.062872,0.034241,0.047211


In [19]:
newtrain.shape

(7613, 602)

In [20]:
newtest.shape

(3263, 601)

In [13]:
from sklearn.preprocessing import MinMaxScaler
from keras.initializers import RandomNormal
from keras.models import Sequential
from keras.layers import Dense, Dropout,BatchNormalization

Using TensorFlow backend.


In [22]:
Xtrain=newtrain.drop(columns=['id','target'])
ytrain=newtrain['target']

In [23]:
Xtestid=newtest['id']
Xtest=newtest.drop(columns=['id'])

In [24]:
mm=MinMaxScaler()

In [25]:
Xtrain=mm.fit_transform(Xtrain)

In [26]:
Xtest=mm.transform(Xtest)

In [27]:
Xtrain.shape

(7613, 600)

In [28]:
model1 = Sequential()
model1.add(Dense(300, input_dim=600, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(100, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(10, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [29]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               180300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
__________

In [32]:
model1.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model1.fit(Xtrain, ytrain,
          epochs=300,
          batch_size=128)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/300
7613/7613 [==============================] - 7s 954us/step - loss: 0.9555 - acc: 0.5225
Epoch 2/300
7613/7613 [==============================] - 1s 91us/step - loss: 0.8830 - acc: 0.5144
Epoch 3/300
7613/7613 [==============================] - 1s 94us/step - loss: 0.8021 - acc: 0.5271
Epoch 4/300
7613/7613 [==============================] - 1s 131us/step - loss: 0.7600 - acc: 0.5240
Epoch 5/300
7613/7613 [==============================] - 1s 121us/step - loss: 0.7352 - acc: 0.5290 0s - loss: 0.7366 - acc: 0.527
Epoch 6/300
7613/7613 [==============================] - 1s 101us/step - loss: 0.7084 - acc: 0.5567 0s - loss: 0.7115 - acc: 0.55
Epoch 7/300
7613/7613 [==============================] - 1s 104us/step - loss: 0.6174 - acc: 0.6779
Epoch 8/300
7613/7613 [==============================] - 1s 90us/step - loss: 0.5619 - acc: 0.7379
Epoch 9/300
7613/7613 [=======================

Epoch 74/300
7613/7613 [==============================] - 1s 102us/step - loss: 0.4681 - acc: 0.7965
Epoch 75/300
7613/7613 [==============================] - 1s 126us/step - loss: 0.4778 - acc: 0.7852
Epoch 76/300
7613/7613 [==============================] - 1s 109us/step - loss: 0.4697 - acc: 0.7940
Epoch 77/300
7613/7613 [==============================] - 1s 125us/step - loss: 0.4668 - acc: 0.7975
Epoch 78/300
7613/7613 [==============================] - 1s 120us/step - loss: 0.4744 - acc: 0.7929 0s - loss: 0.4687 - acc:
Epoch 79/300
7613/7613 [==============================] - 1s 154us/step - loss: 0.4692 - acc: 0.7965
Epoch 80/300
7613/7613 [==============================] - 1s 91us/step - loss: 0.4679 - acc: 0.7954
Epoch 81/300
7613/7613 [==============================] - 1s 90us/step - loss: 0.4674 - acc: 0.7936
Epoch 82/300
7613/7613 [==============================] - 1s 90us/step - loss: 0.4693 - acc: 0.7951
Epoch 83/300
7613/7613 [==============================] - 1s 89us/ste

Epoch 154/300
7613/7613 [==============================] - 1s 87us/step - loss: 0.4484 - acc: 0.8007
Epoch 155/300
7613/7613 [==============================] - 1s 87us/step - loss: 0.4608 - acc: 0.7950
Epoch 156/300
7613/7613 [==============================] - 1s 88us/step - loss: 0.4536 - acc: 0.7999
Epoch 157/300
7613/7613 [==============================] - 1s 86us/step - loss: 0.4552 - acc: 0.7959
Epoch 158/300
7613/7613 [==============================] - 1s 87us/step - loss: 0.4545 - acc: 0.7975
Epoch 159/300
7613/7613 [==============================] - 1s 88us/step - loss: 0.4601 - acc: 0.7935: 0s - loss: 0.4791 - ac
Epoch 160/300
7613/7613 [==============================] - 1s 86us/step - loss: 0.4546 - acc: 0.7929
Epoch 161/300
7613/7613 [==============================] - 1s 88us/step - loss: 0.4599 - acc: 0.7963
Epoch 162/300
7613/7613 [==============================] - 1s 87us/step - loss: 0.4536 - acc: 0.7954
Epoch 163/300
7613/7613 [==============================] - 1s 87us/

In [33]:
newtest.head()

,id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,0.085986,0.017700,0.070801,-0.062646,0.019763,-0.099023,0.024805,-0.098717,0.199609,...,-0.052759,0.013232,-0.023535,0.161035,-0.107227,-0.018237,-0.076904,-0.065820,0.019824,-0.028073
1,2,-0.012672,-0.017273,0.039627,0.100739,-0.048485,-0.000916,-0.074554,-0.013052,0.196361,...,-0.092056,0.001709,-0.069244,0.065786,0.031891,0.020538,-0.036987,-0.052338,-0.011520,-0.035004
2,3,0.093517,0.122688,0.011117,0.044056,-0.021211,-0.021939,-0.050493,-0.093328,0.021833,...,-0.087957,-0.035434,-0.094305,0.111256,0.023299,0.065874,0.019942,-0.051636,0.094627,0.034194
3,9,0.191797,0.008857,-0.048828,0.071875,0.053418,0.051758,-0.064160,-0.124121,0.065332,...,-0.027344,-0.071484,0.004674,0.195703,0.167773,0.001904,-0.104492,-0.031445,0.005469,0.033301
4,11,-0.000142,0.078695,-0.047109,0.059326,-0.040771,-0.005697,-0.017741,-0.138672,0.054911,...,0.069255,-0.004964,-0.101400,0.066895,-0.026286,0.058370,-0.125407,0.075246,0.026042,0.031189


In [34]:
pred=model1.predict(Xtest)

In [35]:
pred[pred>=0.5]=1
pred[pred<0.5]=0

In [36]:
pred=pred.astype('int32')
pred=pd.DataFrame(pred)

In [37]:
submission=pd.DataFrame([Xtestid]).T
submission=pd.concat([submission,pred],axis=1)
submission.columns=['id','target']
submission.set_index('id',inplace=True)
submission.to_csv("Submission.csv")


,target
id,
0,1
2,0
3,1
9,1
11,1
12,1
21,0
22,0
27,0


# LSTM

In [21]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [14]:
Xtrain=train.drop(columns=['id','target'])
ytrain=train['target']
Xtestid=test['id']
Xtest=test.drop(columns=['id'])

In [15]:
max_features = 3000
# cut texts after this number of words (among top max_features most common words)
maxlen = 20
batch_size = 32

In [16]:
train.head()

,id,keyword,location,text,target,hashtags
0,1,NaN,NaN,deeds reason earthquake may allah forgive,1,earthquake
1,4,NaN,NaN,forest fire near ronge sask canada,1,
2,5,NaN,NaN,residents asked shelter place notified officer...,1,
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1,wildfires
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1,alaska wildfires


In [17]:
from keras.preprocessing.text import Tokenizer

In [18]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train.text)

In [19]:
traintext=tokenizer.texts_to_sequences(train.text)
testtext=tokenizer.texts_to_sequences(test.text)

In [22]:
traintext=sequence.pad_sequences(traintext,maxlen=maxlen)
testtext=sequence.pad_sequences(testtext,maxlen=maxlen)

In [23]:
traintext

array([[   0,    0,    0, ...,   50, 1473, 3973],
       [   0,    0,    0, ..., 5945, 5946, 1164],
       [   0,    0,    0, ...,  515, 1165,  894],
       ...,
       [   0,    0,    0, ..., 3686,  447, 1343],
       [   0,    0,    0, ..., 4660, 2265,  181],
       [   0,    0,    0, ...,   75,  581,   10]])

In [24]:
testtext

array([[   0,    0,    0, ..., 1660,   45,   22],
       [   0,    0,    0, ...,  486, 1744,  125],
       [   0,    0,    0, ...,  557, 1175,  228],
       ...,
       [   0,    0,    0, ...,  549,  233, 1086],
       [   0,    0,    0, ...,  133, 1470, 1636],
       [   0,    0,    0, ...,   12,  154, 5436]])

In [25]:
len(tokenizer.word_counts)

14486

In [30]:
model2=Sequential()
model2.add(Embedding(14487,60))
model2.add(LSTM(60, dropout=0.5, recurrent_dropout=0.2,return_sequences=True))
model2.add(LSTM(60, dropout=0.5, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 60)          869220    
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 60)          29040     
_________________________________________________________________
lstm_5 (LSTM)                (None, 60)                29040     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 61        
Total params: 927,361
Trainable params: 927,361
Non-trainable params: 0
_________________________________________________________________


In [32]:
model2.fit(traintext, ytrain,
          batch_size=batch_size,
          epochs=30)

Epoch 1/30
7613/7613 [==============================] - 27s 4ms/step - loss: 0.5702 - acc: 0.6971
Epoch 2/30
7613/7613 [==============================] - 21s 3ms/step - loss: 0.3443 - acc: 0.8570
Epoch 3/30
7613/7613 [==============================] - 22s 3ms/step - loss: 0.2443 - acc: 0.9099
Epoch 4/30
7613/7613 [==============================] - 25s 3ms/step - loss: 0.1829 - acc: 0.9352
Epoch 5/30
7613/7613 [==============================] - 23s 3ms/step - loss: 0.1486 - acc: 0.9473
Epoch 6/30
7613/7613 [==============================] - 22s 3ms/step - loss: 0.1253 - acc: 0.9565
Epoch 7/30
7613/7613 [==============================] - 22s 3ms/step - loss: 0.1077 - acc: 0.9648
Epoch 8/30
7613/7613 [==============================] - 22s 3ms/step - loss: 0.0932 - acc: 0.9665
Epoch 9/30
7613/7613 [==============================] - 21s 3ms/step - loss: 0.0911 - acc: 0.9691
Epoch 10/30
7613/7613 [==============================] - 21s 3ms/step - loss: 0.0801 - acc: 0.9701
Epoch 11/30
7613/76

In [36]:
pred2=model2.predict(testtext)

In [37]:
pred2[pred2>=0.6]=1
pred2[pred2<0.6]=0
pred2=pred2.astype('int32')
pred2=pd.DataFrame(pred2)
submission2=pd.DataFrame([Xtestid]).T
submission2=pd.concat([submission2,pred2],axis=1)
submission2.columns=['id','target']
submission2.set_index('id',inplace=True)
submission2.to_csv("submission2.csv")

In [39]:
submission1=pd.read_csv('Submission.csv')

In [45]:
submission2.reset_index(inplace=True)

In [48]:
newsub=pd.concat([submission1,submission2],axis=1)

In [52]:
newsub.drop(columns=['id'],inplace=True)

In [58]:
newsub=newsub.sum(axis=1)/2

In [59]:
newsub[newsub>=1]=1
newsub[newsub<1]=0

In [62]:
newsub=newsub.astype('int32')
newsub=pd.DataFrame(newsub)
submission3=pd.DataFrame([Xtestid]).T
submission3=pd.concat([submission3,newsub],axis=1)
submission3.columns=['id','target']
submission3.set_index('id',inplace=True)
submission3.to_csv("submission3.csv")